In [54]:
import pandas as pd
df_index= pd.read_excel(r"datasets\(For ML only)CSI300_Index_Lexel(2008).xlsx", engine='openpyxl')
df_index.head()


,date,index_level
0,2008-04-16,3494.02
1,2008-04-17,3386.63
2,2008-04-18,3272.50
3,2008-04-21,3267.55
4,2008-04-22,3296.28


In [55]:
df_index['date']=pd.to_datetime(df_index['date'])
df_index.set_index("date", inplace=True)
weekly_groups=df_index.groupby(pd.Grouper(freq='W'))
#find the last day of each week
df_index_weekly=weekly_groups.last()
df_index_weekly
df_index_weekly.isnull().sum()
df_index_weekly.dropna(inplace=True)

,index_level
date,
2008-04-20,3272.50
2008-04-27,3803.07
2008-05-04,3959.12
2008-05-11,3878.92
2008-05-18,3936.12
...,...
2022-12-04,3870.95
2022-12-11,3998.24
2022-12-18,3954.23


In [56]:
#add index week1 week2 week3...
import numpy as np
df_index_weekly['week'] = range(1, len(df_index_weekly) + 1)
df_index_weekly.set_index("week", inplace=True)
df_index_weekly


,index_level
week,
1,3272.50
2,3803.07
3,3959.12
4,3878.92
5,3936.12
...,...
764,3870.95
765,3998.24
766,3954.23


In [57]:
#last 52 weeks' max value for weekly_indexLevel Dataset
df_index_weekly['max_52'] = None
for i in range(52, len(df_index_weekly)+1):
    previous_52 = df_index_weekly.loc[i-52:i-1, 'index_level']  
    max_value = previous_52.max()  
    df_index_weekly.at[i, 'max_52'] = max_value  

df_index_weekly

,index_level,max_52
week,,
1,3272.50,None
2,3803.07,None
3,3959.12,None
4,3878.92,None
5,3936.12,None
...,...,...
764,3870.95,5055.12
765,3998.24,5055.12
766,3954.23,4954.76


In [58]:
#calculate ML = index_level/(max of last 52 weeks) and create a new column
df_index_weekly['ML'] = df_index_weekly['index_level']/df_index_weekly['max_52']
df_index_weekly

,index_level,max_52,ML
week,,,
1,3272.50,None,NaN
2,3803.07,None,NaN
3,3959.12,None,NaN
4,3878.92,None,NaN
5,3936.12,None,NaN
...,...,...,...
764,3870.95,5055.12,0.765748
765,3998.24,5055.12,0.790929
766,3954.23,4954.76,0.798067


In [59]:
#find the mean and standard deviation of ML for past 52 weeks
df_index_weekly['mean_52'] = None
df_index_weekly['std_52'] = None
for i in range(104, len(df_index_weekly)+1):
    previous_52 = df_index_weekly.loc[i-52:i-1, 'ML']  
    mean_value = previous_52.mean()  
    std_value = previous_52.std()  
    df_index_weekly.at[i, 'mean_52'] = mean_value  
    df_index_weekly.at[i, 'std_52'] = std_value

df_index_weekly

,index_level,max_52,ML,mean_52,std_52
week,,,,,
1,3272.50,None,NaN,None,None
2,3803.07,None,NaN,None,None
3,3959.12,None,NaN,None,None
4,3878.92,None,NaN,None,None
5,3936.12,None,NaN,None,None
...,...,...,...,...,...
764,3870.95,5055.12,0.765748,0.799426,0.041597
765,3998.24,5055.12,0.790929,0.797864,0.04136
766,3954.23,4954.76,0.798067,0.796274,0.040021


In [60]:
df_index_weekly['mean-2.5*std'] = df_index_weekly['mean_52'] - 2.5*df_index_weekly['std_52']
df_index_weekly['CC'] = np.where(df_index_weekly['ML'] < df_index_weekly['mean-2.5*std'], 1, 0)
df_index_weekly

,index_level,max_52,ML,mean_52,std_52,mean-2.5*std,CC
week,,,,,,,
1,3272.50,None,NaN,None,None,NaN,0
2,3803.07,None,NaN,None,None,NaN,0
3,3959.12,None,NaN,None,None,NaN,0
4,3878.92,None,NaN,None,None,NaN,0
5,3936.12,None,NaN,None,None,NaN,0
...,...,...,...,...,...,...,...
764,3870.95,5055.12,0.765748,0.799426,0.041597,0.695433,0
765,3998.24,5055.12,0.790929,0.797864,0.04136,0.694463,0
766,3954.23,4954.76,0.798067,0.796274,0.040021,0.696222,0


In [61]:
#to csv
df_index_weekly.to_csv(r'datasets\CSI300_IL_Weekly_ML.csv', index = True, header=True)

In [62]:
df_index_weekly.describe()

,index_level,CC
count,753.000000,768.000000
mean,3388.940890,0.035156
std,852.227463,0.184295
min,1663.660000,0.000000
25%,2647.100000,0.000000
50%,3340.120000,0.000000
75%,3904.940000,0.000000
max,5807.720000,1.000000
